In [1]:
import pandas as pd
from IPython.display import Image
from six import StringIO
import pydotplus

In [2]:
dataset=pd.read_csv('C:/Users/tengd/OneDrive/0-论文/dataset.csv')

In [3]:
range_x=dataset['WGS84X'].max()-dataset['WGS84X'].min()
range_y=dataset['WGS84Y'].max()-dataset['WGS84Y'].min()
print(range_x)
print(range_y)

0.8686197000000107
0.6369052199999992


In [4]:
data=pd.DataFrame(dataset,columns=['price202403','WGS84X','WGS84Y','county'])
# data['price202403']=data['price202403']*0.000006
countys=list(data['county'].unique())
data_list=[]
data_location=[]
data_other=[]
for county in countys:
    temp=data.loc[data['county']==county]
    data_list.append(temp.drop(columns=['WGS84X','WGS84Y','county']))
    data_location.append(temp.drop(columns=["price202403",'county']))
    data_other.append(temp.drop(columns=["price202403"]))

In [5]:
import numpy as np
import scipy.sparse
data_matrix_list=[]
data_location_list=[]
for data in data_list:
    data_matrix=scipy.sparse.csr_matrix(data.values)
    data_matrix_list.append(data_matrix)
for data in data_location:
    location=scipy.sparse.csr_matrix(data.values)
    data_location_list.append(location)

In [6]:
from sklearn.cluster import KMeans

In [7]:
for i in range(len(data_matrix_list)):
    data_matrix=data_matrix_list[i]
    label=KMeans(n_clusters=2,n_init=100).fit_predict(data_matrix)
    label_location=KMeans(n_clusters=5,n_init=100).fit_predict(data_location_list[i])
    data_list[i]['label_price']=label
    data_list[i]['label_loc']=label_location
    data_list[i]['WGS84X']=data_other[i]['WGS84X']
    data_list[i]['WGS84Y']=data_other[i]['WGS84Y']
    data_list[i]['county']=data_other[i]['county']
    print(data_list[i])

      price202403  label_price  label_loc      WGS84X     WGS84Y county
0            6977            1          0  114.001108  30.582167     蔡甸
156          7800            1          3  114.081796  30.565280     蔡甸
157         11616            1          1  114.092301  30.515956     蔡甸
164          8138            1          0  114.003647  30.577078     蔡甸
171          6640            0          1  114.107297  30.519709     蔡甸
...           ...          ...        ...         ...        ...    ...
5981         6573            0          0  114.013223  30.593234     蔡甸
5982         5229            0          0  114.015303  30.578107     蔡甸
5983         7202            1          0  114.024773  30.576619     蔡甸
5984         4461            0          0  114.017281  30.589010     蔡甸
5985         6082            0          0  114.048706  30.567943     蔡甸

[277 rows x 6 columns]
      price202403  label_price  label_loc      WGS84X     WGS84Y county
1            6565            0          

In [8]:
import matplotlib.pyplot as plt
df=pd.concat(data_list,axis=0)
df.to_csv('C:/Users/tengd/OneDrive/0-论文/result_simpleKMeans_onlyPrice.csv',index=False)
# data_matrix.shape